In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, Dense, Embedding, Flatten, Input, LSTM, MaxPooling1D
from keras.utils import np_utils
from keras.models import Model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [3]:
%store -r data
data.head()

,sentiment,text
1,positive,Greetings and welcome to the Microsoft Fiscal ...
10,positive,technology. Microsoft 365 helps every organiza...
100,positive,is coming from the line of Walter Pritchard wi...
101,neutral,we're growing at eye-popping rates right now. ...
102,positive,the key things that we think about is differen...


In [4]:
texts = data.text
labels = data.sentiment.map(str.lower).map({'positive' : 1, 'neutral' : 0, 'negative' : 2})

In [5]:
texts, X_test, labels, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [6]:
len(texts)

497

In [7]:
maxlen = 400  # We will cut reviews after 400 words
training_samples = 400  # We will be training on 200 samples
validation_samples = 97  # We will be validating on 10000 samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Found 3345 unique tokens.
Shape of data tensor: (497, 400)
Shape of label tensor: (497,)


In [8]:
embeddings_index = {}
f = open('glove.6B.100d.txt', encoding="utf8")

In [9]:
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [11]:
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [22]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 400, 100)          1000000   
_________________________________________________________________
flatten_4 (Flatten)          (None, 40000)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 1000)              40001000  
_________________________________________________________________
dense_10 (Dense)             (None, 100)               100100    
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 41,101,201
Trainable params: 41,101,201
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [24]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=5,
                    batch_size=40,
                    validation_data=(x_val, y_val))

Train on 400 samples, validate on 97 samples
Epoch 1/5
400/400 [==============================] - 6s 16ms/step - loss: 2.0763 - acc: 0.5050 - val_loss: 0.1828 - val_acc: 0.5567
Epoch 2/5
400/400 [==============================] - 6s 15ms/step - loss: 1.0803 - acc: 0.6125 - val_loss: 0.5516 - val_acc: 0.4948
Epoch 3/5
400/400 [==============================] - 6s 15ms/step - loss: 0.1611 - acc: 0.6900 - val_loss: 1.4254 - val_acc: 0.5464
Epoch 4/5
400/400 [==============================] - 6s 15ms/step - loss: 0.0123 - acc: 0.7025 - val_loss: 0.4159 - val_acc: 0.5464
Epoch 5/5
400/400 [==============================] - 6s 15ms/step - loss: -0.6928 - acc: 0.7875 - val_loss: 0.6524 - val_acc: 0.5464


In [26]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_test)
sequences = tokenizer.texts_to_sequences(X_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X_test = pad_sequences(sequences, maxlen=maxlen)

Found 1747 unique tokens.


In [27]:
y_pred = model.predict(X_test)

In [28]:
from sklearn.metrics import confusion_matrix

In [32]:
y_pred = y_pred.round()

In [34]:
print(confusion_matrix(y_pred=y_pred, y_true=y_test))

[[ 8 44  0]
 [ 1 59  0]
 [ 0 13  0]]
